<a href="https://colab.research.google.com/github/jacqueslethuaut/petals_to_the_metals/blob/main/petals_to_the_metals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Petals to the Metal - Flower Classification on TPU

Unranked kaggle competition

Author : Jacques LE THUAUT

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os, zipfile

os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/Kaggle'

ROOT_DIR = '/content/drive/MyDrive/Colab Notebooks/'
ORIGINAL_IMAGES_DIR = '/content/flower_images/'

In [5]:
%mkdir $ORIGINAL_IMAGES_DIR

In [6]:

!kaggle competitions download -c tpu-getting-started -p $ORIGINAL_IMAGES_DIR

100% 4.78G/4.79G [00:51<00:00, 128MB/s]
100% 4.79G/4.79G [00:51<00:00, 100MB/s]


In [8]:
path = ORIGINAL_IMAGES_DIR+'tpu-getting-started.zip'

# The data is provided as a zip file so we need to extract the files from the zip file
with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall()

In [9]:
import math, re
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers, models, applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121

from sklearn.metrics import classification_report



In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:


GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(GCS_DS_PATH) # what do gcs paths look like?

In [ ]:
PATH_TO_FLOWERS =

In [ ]:
image_size = (224, 224)
batch_size = 32

# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'path/to/flower/dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'path/to/flower/dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
base_model = applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
predictions = layers.Dense(5, activation='softmax')(x)


model = models.Model(inputs=base_model.input, outputs=predictions)
